###**Automating VBA Macro Documentation and Transformation**

**Team:**  
*   **20PD02** - Aditya Ramanathan
*   **20PD06** - Ashwin KR



**Objective:**  

*   The goal is to develop a solution that automates the documentation and understanding of legacy VBA macros used in critical processes.

In [ ]:
pip install --quiet oletools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 989.4/989.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.0 MB/s eta 0:00:00


In [ ]:
pip install --quiet langchain langchain-core langchain-community langchain-groq

In [ ]:
#import necessary libraries

import os
import re
from oletools.olevba import VBA_Parser

from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def extract_vba_from_excel(file_path):
    '''
    Args:
      file_path (str): Location to the Excel Workbook
    Returns:
      code (str): The VBA Macro extracted from the Workbook (if available else None)
    '''
    vba_parser = VBA_Parser(file_path)

    if vba_parser.detect_vba_macros():
      vba_modules = vba_parser.extract_all_macros()
    else:
      print("No Macros found in the Given Workbook!")
      return ""

    code = ""
    for _, _, vba_name, vba_code in vba_modules:
      if vba_name.endswith('.bas'):
        code += vba_code + '\n'

    if code:
      return code
    else:
      print("The Macros do not contain any written code!")
      return ""

def preprocess_code(vba_code):
    '''
    Args:
      vba_code (str): The VBA Macro code extracted from the Workbook
    Returns:
      vba_code_processed (str): The VBA Macro code after processing
    '''
    vba_code = vba_code.replace('\r', ' ').replace('\t', ' ')

    _RE_COMBINE_WHITESPACE = re.compile(r"\s+") #strip multiple whitespaces
    vba_code_processed = _RE_COMBINE_WHITESPACE.sub(" ", vba_code).strip()

    return vba_code_processed


In [ ]:
excel_file_path = '/content/security_test.xlsm'

vba_code_unprocessed = extract_vba_from_excel(excel_file_path)
vba_code = preprocess_code(vba_code_unprocessed)

print(vba_code)

Attribute VB_Name = "Module1" Sub security_test() Dim con As Object Dim rs As Object Dim username As String Dim password As String username = InputBox("Enter username:") password = InputBox("Enter password:") Set con = CreateObject("ADODB.Connection") Set rs = CreateObject("ADODB.Recordset") con.Open "Provider=SQLOLEDB;Data Source=MyDB;User Id=admin;Password=admin;" Dim sql As String sql = "INSERT INTO Users (username, password) VALUES ('" & username & "', '" & password & "');" con.Execute sql con.Close End Sub


In [ ]:
def initialize_llm_via_groq(api_key, model):
  '''
  Args:
    api_key (str): API key to access models via GroqCloud
    model (str)  : Name of the Large Language Model (LLM) to be used
  Returns:
    llm (langchain_groq.chat_model) : The intitalized LLM
  '''

  llm = ChatGroq(
        temperature = 0,
        model = model,
        api_key = api_key
       )

  return llm

In [ ]:
model="mixtral-8x7b-32768"
api_key = "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"

llm = initialize_llm_via_groq(api_key, model)

**Automated VBA Macro Documentation**

*   Automatically analyzes Excel VBA Macros and generates a comprehensive documentation of the underlying logic, data flow and process flow.




In [ ]:
prompt_template_vba_macro_documentation="""
You are a helpful assistant.
You are responsible for analyzing the given VBA Macro code and generating a comprehensive documentation of the underlying logic, data flow and process flow.
Your response must definitely contain the underlying logic, data flow and process flow for each of the subroutines available in the given code.
Avoid stopwords, special characters or escape sequences in your response. Ignore warning statements in the code if any.
VBA Code:{question}
"""

prompt=PromptTemplate(template=prompt_template_vba_macro_documentation)

query_with_prompt = prompt.format(question = vba_code)

In [ ]:
vba_macro_documentation = llm.invoke(query_with_prompt).content
vba_macro_documentation.replace('\n', ' ')

'Sure, I\'d be happy to help you understand the VBA macro code provided. Here\'s a comprehensive documentation of the underlying logic, data flow, and process flow for the `security_test` subroutine:  **Subroutine Name:** security\\_test  **Purpose:** This subroutine prompts the user to enter a username and password, and then attempts to insert those values into a SQL database table called "Users".  **Logic:**  1. The subroutine begins by declaring variables for the database connection object (`con`), the database recordset object (`rs`), the username (`username`), and the password (`password`). 2. The `username` and `password` variables are populated with values entered by the user in response to `InputBox` prompts. 3. A new database connection object (`con`) is created using the `CreateObject` method and the connection string for a SQL database called "MyDB". The connection string includes the username and password for a database user with administrative privileges. 4. A new database

**Functional Logic Extractor**


*   Extracts and explains the functional logic embedded within the VBA Macros



In [ ]:
prompt_template_functional_logic="""
You are a helpful assistant.
You are responsible for extracting and explaining the functional logic embedded within the given VBA macro.
Your response should help technical and non-technical stakeholders understand the business logic, supporting better decision-making
and transformation efforts.
Structure your response using '~', splitting the response wherever necessary.
Avoid stopwords, special characters or escape sequences in your response. Ignore warning statements in the code if any.
VBA Code:{question}
"""

prompt=PromptTemplate(template=prompt_template_functional_logic)

query_with_prompt = prompt.format(question = vba_code)

In [ ]:
vba_macro_functional_logic = llm.invoke(query_with_prompt).content
vba_macro_functional_logic.replace('\n', ' ')

'Functional Logic:  ~ Extracts and explains the functional logic within the VBA macro named "security\\_test"  ~ Three main functional components:   1. Data Input: Username and Password input via InputBox   2. Database Connection: ADODB Objects for Connection and Recordset   3. Data Insertion: SQL INSERT statement to add user data into the \'Users\' table  Data Input: ~ Username and password are obtained through InputBox prompts ~ User input is stored in the variables "username" and "password"  Database Connection: ~ ADODB Objects for Connection and Recordset are created ~ Connection is established using a predefined connection string   - Provider: SQLOLEDB   - Data Source: MyDB   - User Id: admin   - Password: admin  Data Insertion: ~ SQL INSERT statement is constructed using the input variables "username" and "password" ~ The INSERT statement is executed using the "con.Execute" method ~ Connection is closed using the "con.Close" method  Implications: ~ This macro allows for the addit

**Automated Process Flow Visualization**

*   Visualize the process flow of VBA Macros through interactive diagrams.



In [ ]:
def extract_nodes_and_links(vba_code):
    '''
    Args:
      vba_code (str): The VBA Macro code extracted
    Returns:
      nodes (list[str]): The subroutines forming the nodes in the process flow
      links (list[str]): The calls between subroutines forming the links between nodes
    '''
    nodes = []
    links = []

    #define regex patterns to match subroutine names and calls
    subroutine_pattern = r'Sub\s+(\w+)\('
    call_pattern = r'Call\s+(\w+)\b'

    #find all subroutine names (nodes in the process flow)
    subroutine_matches = re.findall(subroutine_pattern, vba_code)
    nodes.extend(subroutine_matches)

    #find all calls between subroutines (links between nodes)
    for match in re.finditer(subroutine_pattern, vba_code):
        subroutine_name = match.group(1)
        start = match.end()
        end = re.search(r'End\s+Sub', vba_code[start:]).start()
        subroutine_body = vba_code[start:start+end]
        call_matches = re.findall(call_pattern, subroutine_body)
        for call in call_matches:
            if call in nodes:
                links.append((subroutine_name, call))

    return nodes, links

nodes, links = extract_nodes_and_links(vba_code)

print("Nodes:", nodes)
print("Links:", links)

Nodes: ['main', 'init', 'setupEnvironment', 'loadData', 'fetchFromDatabase', 'readFromFile', 'processData', 'transformData', 'analyzeData', 'generateReport', 'finalize', 'saveResults', 'cleanup']
Links: [('main', 'init'), ('main', 'processData'), ('main', 'finalize'), ('init', 'setupEnvironment'), ('init', 'loadData'), ('loadData', 'fetchFromDatabase'), ('loadData', 'readFromFile'), ('processData', 'transformData'), ('processData', 'analyzeData'), ('analyzeData', 'generateReport'), ('finalize', 'saveResults'), ('finalize', 'cleanup')]


In [ ]:
nodes_js = ",\n".join([f"{{ id: '{node}' }}" for node in nodes])
links_js = ",\n".join([f"{{ source: '{link[0]}', target: '{link[1]}' }}" for link in links])

with open('data.js', 'w') as f:
    f.write(f"const nodes = [\n{nodes_js}\n];\n\nconst links = [\n{links_js}\n];\n")

print("Wrote to data.js") #to be used in streamlit app to visualize process flow

Wrote to data.js


**Code Quality and Efficiency Analyzer**


*   Evaluates the quality and efficiency of VBA Macros, identifying potential optimization opportunities to improve performance.


In [ ]:
prompt_template_code_quality="""
Please analyze the provided VBA Macro code and evaluate its quality and efficiency.
Identify potential inefficiencies, redundant code, and optimization opportunities to improve macro performance and reliability.
Pay particular attention in analyzing:
  Code Efficiency: Assess if the code is optimized for speed and resource usage.
  Redundant Code: Identify any sections of code that repeat functionality unnecessarily.
  Optimization Opportunities: Suggest improvements or optimizations that could enhance the macro's performance.
Please provide only a detailed feedback on each of these aspects to guide improvements in the given VBA Macro code.
Do not provide any VBA code as a part of the response.
Here is the VBA Macro code snippet:
VBA Code:{question}
"""

prompt=PromptTemplate(template=prompt_template_code_quality)

query_with_prompt = prompt.format(question = vba_code)


In [ ]:
vba_macro_code_quality = llm.invoke(query_with_prompt).content
vba_macro_code_quality.replace('\n', ' ')

'Code Efficiency: The code could be more efficient by using early binding instead of late binding for the ADODB objects. Early binding provides IntelliSense and type checking, which can help catch errors at compile time rather than at runtime. Additionally, early binding is faster than late binding because it avoids the overhead of runtime object creation. To use early binding, you would need to set a reference to the Microsoft ActiveX Data Objects Library in the VBA editor (Developer tab -> References).  Redundant Code: The code does not contain any obvious redundancies. However, it is generally a good practice to validate user input, such as checking if the username and password fields are empty or contain only whitespace. This would help prevent potential errors and improve the overall robustness of the code.  Optimization Opportunities: 1. Use parameterized queries instead of concatenating SQL strings to prevent SQL injection attacks and improve performance. Parameterized queries c

**Data Flow Analysis Optimization**


*   Analyzes the data flow within the VBA Macros to enhance the efficiency and performance of data processing tasks.



In [ ]:
prompt_template_data_flow="""
Please analyze the provided VBA Macro code and evaluate the data flow within the macros.
Identify bottlenecks and opportunities for optimization to enhance efficiency and performance in data processing tasks. Specifically, focus on:
Data Flow Analysis: Map out how data moves through the macro from input to output.
Bottlenecks: Identify points in the code where data processing slows down or encounters inefficiencies.
Optimization Opportunities: Recommend changes or optimizations to streamline data processing and improve overall performance.
Resource Usage: Assess how efficiently resources are utilized during data processing.
Scalability: Consider how well the macro handles varying data volumes and complexity.

Please provide detailed insights and recommendations to optimize the VBA Macro code for improved efficiency and performance in data processing tasks.
Here is the VBA Macro code snippet:
VBA Code:{question}
"""

prompt=PromptTemplate(template=prompt_template_data_flow)

query_with_prompt = prompt.format(question = vba_code)


In [ ]:
vba_macro_data_flow = llm.invoke(query_with_prompt).content
vba_macro_data_flow.replace('\n', ' ')

"Data Flow Analysis: In the `security_test` subroutine, data flows as follows: 1. The `username` and `password` variables are initialized with values entered by the user through the `InputBox` function. 2. The `ADODB.Connection` and `ADODB.Recordset` objects are created and assigned to the `con` and `rs` variables, respectively. 3. The `con` object's `Open` method is called with a connection string to establish a connection to the database. 4. The `sql` variable is assigned a string containing an SQL `INSERT` statement that uses the `username` and `password` variables as values. 5. The `con` object's `Execute` method is called with the `sql` string to execute the SQL statement, inserting the user-entered data into the `Users` table. 6. The `con` object's `Close` method is called to close the database connection.  Bottlenecks: The primary bottleneck in this code is the use of the `InputBox` function to gather user input. This function requires user interaction, which can be slow and pro

**Legacy Macro Modernization Assistant**


*   Assists in the modernization of VBA Macros by providing recommendations for refactoring or rewriting code



In [ ]:
prompt_template_refactor="""
Please provide recommendations for refactoring or rewriting the given VBA Macro code (only if necessary) using modern programming languages and technologies to
enhance maintainability, scalability, and performance.
Consider the following:
  Language and Technology Recommendations: Suggest suitable modern programming languages (e.g., Python) and technologies (e.g., APIs) that align with the macro's functionality.
  Refactoring Strategies: Recommend specific refactoring techniques to improve code structure, readability, and maintainability.
  Integration Possibilities: Explore integration options with other systems or platforms for enhanced functionality and interoperability.
  Performance Optimization: Identify opportunities to optimize code performance and efficiency in the new environment.
  Compatibility Considerations: Address compatibility issues or considerations when transitioning from VBA to modern languages or technologies.

Please provide detailed guidance on how to effectively modernize the VBA Macro code, ensuring it meets current industry standards and best practices.

Here is the VBA Macro code snippet:
VBA Code:{question}
"""

prompt=PromptTemplate(template=prompt_template_refactor)

query_with_prompt = prompt.format(question = vba_code)


In [ ]:
vba_macro_refactor = llm.invoke(query_with_prompt).content
vba_macro_refactor.replace('\n', ' ')

'I recommend refactoring the VBA macro code using Python, a modern programming language, and SQLAlchemy, a popular Object-Relational Mapping (ORM) tool for working with databases. This approach will enhance maintainability, scalability, and performance while addressing compatibility considerations.  Language and Technology Recommendations: - Python: A modern, high-level programming language with a large community, extensive libraries, and excellent support for database operations. - SQLAlchemy: A popular ORM tool for Python that simplifies database interactions and provides a more Pythonic interface for working with databases.  Refactoring Strategies: 1. Use a connection pool: SQLAlchemy provides a connection pool that manages connections efficiently, reducing overhead and improving performance. 2. Parameterize SQL queries: Prevent SQL injection attacks and improve code readability by using parameterized queries. 3. Use context managers for resource management: Automatically close conn

**Security and Compliance Checker**


*   Checks VBA Macros for security vulnerabilities and adherance to best practices.



In [ ]:
def check_risky_patterns(vba_code):
    '''
    Args:
      vba_code (str): The extracted VBA Macro code
    Returns:
      risks_found (list[str]): High risk keywords found in the VBA code
    '''

    risky_patterns = [
        r'\bShell\b',
        r'\bExecute\b',
        r'\bActiveX\b'
    ]

    risks_found = []
    for pattern in risky_patterns:
        matches = re.findall(pattern, vba_code, re.IGNORECASE)
        if matches:
            risks_found.extend(matches)

    return risks_found

def check_error_handling(vba_code):
    '''
    Args:
      vba_code (str): The extracted VBA Macro code
    Returns:
      True: If error handling feature available in VBA code
    '''
    if "On Error" not in vba_code:
        return False
    return True

def sanitize_user_inputs(vba_code):
    '''
    Args:
      vba_code (str): The extracted VBA Macro code
    Returns:
      sanitization_issues (list[str]): SQL patterns that are vulnerable to injection attacks
    '''

    sql_patterns = [
        r'INSERT INTO',
        r'SELECT \* FROM',
        r'UPDATE .* SET',
        r'DELETE FROM'
    ]

    sanitization_issues = []
    for pattern in sql_patterns:
        if re.search(pattern, vba_code, re.IGNORECASE):
            if "'" in vba_code:
                sanitization_issues.append(pattern)

    return sanitization_issues


In [ ]:
risky_functions = check_risky_patterns(vba_code)
error_handling = check_error_handling(vba_code)
sanitization_issues = sanitize_user_inputs(vba_code)

#report findings
if risky_functions:
    print("Risky Patterns Found:")
    for func in risky_functions:
        print(f" - {func}")
    print("The above listed functions pose a security risk. It may execute external applications, which could be exploited by attackers.\n")
else:
  print("No risky patterns identified in the given VBA Macro.\n")

if not error_handling:
    print("Error Handling: \nNot present. Proper error handling is crucial to prevent the system from exposing sensitive information during errors and to ensure that the application behaves securely and predictably.\n")

if sanitization_issues:
    print("Sanitization Issues Found:")
    for issue in sanitization_issues:
        print(f" - {issue}")
    print("The above listed SQL patterns may be vulnerable to SQL injection if not properly parameterized. Ensure that SQL queries are constructed using parameterized queries or stored procedures to prevent injection attacks.\n")
else:
    print("All user inputs are properly sanitized, reducing the risk of injection attacks.\n")

Risky Patterns Found:
 - Execute
The above listed functions pose a security risk. It may execute external applications, which could be exploited by attackers.

Error Handling: 
Not present. Proper error handling is crucial to prevent the system from exposing sensitive information during errors and to ensure that the application behaves securely and predictably.

Sanitization Issues Found:
 - INSERT INTO
The above listed SQL patterns may be vulnerable to SQL injection if not properly parameterized. Ensure that SQL queries are constructed using parameterized queries or stored procedures to prevent injection attacks.

